In [23]:
import pandas as pd
import matplotlib.pyplot as plt
import warnings

warnings.filterwarnings("ignore")
plt.style.use("dark_background")
random_state=42

In [24]:
tesis = pd.read_csv("../Datos/tesis_final.csv")
tesis = tesis[tesis["Mes"] <= 12]
tesis["Intercept"] = 1

In [25]:
tesis_lag = tesis[["idPaciente", "Intercept", "TAS", "Mes", "Adherencia", "Adherencia_Acumulada", "TAS_Media_Acumulada"]]
lagged_vars = ["Adherencia", "TAS", "Adherencia_Acumulada", "TAS_Media_Acumulada"]
for var in lagged_vars:
    for i in range(1, 4):
        tesis_lag[f"{var}_lag{i}"] = tesis_lag.groupby("idPaciente")[var].shift(i)

In [26]:
max_range = tesis["Mes"].max()+1

# Modelos para cada tiempo con 1 variables rezagada y promedio acumulado

In [27]:
# Exogeneidad por tiempo con un solo lag
from statsmodels.discrete.discrete_model import Logit
tiempos = []
for i in range(3, max_range):
    tesis_aux = tesis_lag[tesis_lag["Mes"] == i]
    print("\n", "*"*50, "Target time:", i, "*"*50, "\n")
    fixed_effects = ["Intercept", "Adherencia_lag1", "Adherencia_Acumulada_lag2", "TAS_lag1", "TAS_Media_Acumulada_lag2"]
    lr = Logit(tesis_aux["Adherencia"], tesis_aux[fixed_effects], groups=tesis_aux["idPaciente"]).fit()
    print(lr.summary())
    if (lr.pvalues["TAS_lag1"] < 0.05) or (lr.pvalues["TAS_Media_Acumulada_lag2"] < 0.05):
        tiempos.append(str(i))


 ************************************************** Target time: 3 ************************************************** 

Optimization terminated successfully.
         Current function value: 0.369976
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:             Adherencia   No. Observations:                  560
Model:                          Logit   Df Residuals:                      555
Method:                           MLE   Df Model:                            4
Date:                Wed, 07 Jun 2023   Pseudo R-squ.:                  0.2115
Time:                        20:03:46   Log-Likelihood:                -207.19
converged:                       True   LL-Null:                       -262.76
Covariance Type:            nonrobust   LLR p-value:                 4.131e-23
                                coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------

In [28]:
print(f"De los modelos para cada tiempo con un solo lag en las covariables, la variables TAS es significativa en el/los tiempos {', '.join(tiempos)}")

De los modelos para cada tiempo con un solo lag en las covariables, la variables TAS es significativa en el/los tiempos 
